# BERT Question Answering Demo

This notebooks demonstrates a Question Answering demo application that uses a SQuAD-tuned BERT model from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/index.md#question-answering). 
See `bert_notebook_utils.py` in the same folder as this notebook for the source code to load the model and perform inference. 

The demo notebook loads a BERT network to [OpenVINO](https://github.com/openvinotoolkit/openvino) Inference Engine. It also fetches data from a user-provided url to populate the "context" text.
The text is then used to search answers for user-provided questions.

## Settings and Imports

Change the `input_url` to change the context of the model.

In [ ]:
import os
import subprocess
import sys

from bert_notebook_utils import BERT

In [ ]:
input_url = "https://en.wikipedia.org/wiki/Bert_(Sesame_Street)"

# Other settings (only change this if you know what you are doing!)
model_name = "bert-small-uncased-whole-word-masking-squad-0001"
device = "CPU"
reshape = False  # Try to reshape the sequence length to the input context + max question len (to improve the speed)
model_squad_ver = "1.2"  # SQuAD version used for model fine tuning

In [ ]:
# Set file and directory paths for model. By default, this demo notebook downloads all BERT models from the Open Model Zoo
# and stores them in the current directory. Adjust these settings if you want to change this.
# !curl -O https://raw.githubusercontent.com/openvinotoolkit/open_model_zoo/master/models/intel/bert-small-uncased-whole-word-masking-squad-0001/vocab.txt
vocab_file = "vocab.txt"  # vocab file for all BERT models
base_model_dir = os.curdir  # Models will be downloaded into the `intel` folder in this directory
omz_cache_dir = os.path.expanduser("~/open_model_zoo_cache")

## Download BERT models

Download BERT models from Open Model Zoo with the Model Downloader. Models are downloaded to `base_model_dir`, which is set to the current directory by default. Open Model Zoo caches the downloaded models in `omz_cache_dir`. By default this is set to `open_model_zoo_cache` in the home directory (`/home/username` on Linux, `c:\Users\username` on Windows). If you want to modify these defaults, change the settings in the previous cell. 

Supported models from the Open Model Zoo are:

* bert-large-uncased-whole-word-masking-squad-0001
* bert-large-uncased-whole-word-masking-squad-int8-0001
* bert-small-uncased-whole-word-masking-squad-0001
* bert-small-uncased-whole-word-masking-squad-0002
* bert-small-uncased-whole-word-masking-squad-int8-0002

See the [Open Model Zoo documentation](https://github.com/openvinotoolkit/open_model_zoo/blob/master/tools/downloader/README.md) for more information on the Model Downloader and [Open Model Zoo Models](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/index.md#question-answering) for information about the models.

The code in the next cell downloads the "small" models from Open Model Zoo. Change `downloader_model_name` to `"bert*"` to download all BERT models, or to a specific model name, for example `bert-large-uncased-whole-word-masking-squad-0001` to download only that model.

In [ ]:
downloader_model_name = "bert-small*"
! omz_downloader --name $downloader_model_name --jobs 4 --cache_dir $omz_cache_dir --precision "FP16,FP16-INT8" --output_dir $base_model_dir


## Setup BERT

In [ ]:
bert = BERT(
    input_url, vocab_file, model_name, base_model_dir, reshape=reshape, device=device, model_squad_ver=model_squad_ver
)

## Ask Questions!

The BERT model returns the answer, a score (the higher the score, the more confident the model is), and the context that was used to find the answer.

In [ ]:
bert.ask("What is BERT?")

# OPTIONAL

The functions above are all you need to get started with a BERT model from the Open Model Zoo. If you want to experiment more, the following cells have some suggestions. 

## Change the input URL and ask more questions

In [ ]:
bert.set_input_url("https://en.wikipedia.org/wiki/BERT_(language_model)")
bert.ask(["What is BERT?", "Who made BERT?", "What prize did BERT win?"])

If you just want to see the answers, you can hide the context.

In [ ]:
bert.ask("What is BERT?", show_context=False)

In [ ]:
bert.set_input_url("https://en.wikipedia.org/wiki/Intel")
bert.ask(["Who is the CEO?", "How many people work there?", "How much profit does Intel make?"])

## Change the BERT model

Select a model name from these models that you download in the *Download BERT models* section. Use `bert.set_model_name()` to set the name of the model you want to try. Note that the models with `large` in the name require more than 2GB of memory and may not run on Binder.

In [ ]:
bert.set_model_name("bert-small-uncased-whole-word-masking-squad-int8-0002")
bert.ask(["What is BERT?", "Who made BERT?", "What prize did BERT win?"])